<a href="https://colab.research.google.com/github/VasireddyNandini/predictive-analysis/blob/book_recommend/book%20recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
df = pd.read_csv("Books.csv", encoding='latin-1', on_bad_lines='skip', sep=';')
df['Book-Title'] = df['Book-Title'].fillna('').str.lower()
df['Book-Author'] = df['Book-Author'].fillna('').str.lower()
df['Publisher'] = df['Publisher'].fillna('').str.lower()
df['Combined'] = df['Book-Title'] + ' ' + df['Book-Author'] + ' ' + df['Publisher']
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['Combined'])
model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
model.fit(tfidf_matrix)
title_to_index = pd.Series(df.index, index=df['Book-Title']).drop_duplicates()
def recommend_books_by_metadata(title, top_n=1):
    title = title.lower()
    if title not in title_to_index.index:
        return f"Book '{title}' not found in dataset."

    idx = title_to_index[title]
    query_vector = tfidf_matrix[idx]
    distances, indices = model.kneighbors(query_vector, n_neighbors=top_n + 1)
    recommended_indices = indices.flatten()[1:]
    result = df.iloc[recommended_indices][['Book-Title', 'Book-Author']].drop_duplicates()
    result['Book-Title'] = result['Book-Title'].str.title().str.slice(0, 50)  # Optionally truncate
    result['Book-Author'] = result['Book-Author'].str.title()
    return result.reset_index(drop=True)
recommendations = recommend_books_by_metadata("The Alchemist", top_n=1)
print(recommendations)

/tmp/ipython-input-18-773872406.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Books.csv", encoding='latin-1', on_bad_lines='skip', sep=';')


                                          Book-Title             Book-Author
0                                               Prey             Ken Goddard
1                                      The Alchemist              Mark Illis
2           A Story For Europe (Bloomsbury Classics)               Will Self
3                                      The Alchemist              Donna Boyd
4                                      The Alchemist              Ben Jonson
5   The Alchemist And Other Plays (World'S Classics)              Ben Jonson
6                                      The Alchemist  Jorge A. Livraga Rizzi
7                              The Spirits Of Nature        Jorge A. Livraga
8                                      The Alchemist       Leslie H. Whitten
9  The Fangs Of The Morning/The Alchemist/2 Compl...       Leslie H. Whitten
